In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# !pip install fuzzywuzzy
from fuzzywuzzy import fuzz

# Exploring Database Tables

In [ ]:
-- Execute this cell to return the list of tables
-- and views within the Fossils schema of this database
-- (Use \help to learn more options)

\list Fossils.*

In [ ]:
-- Execute this cell to return the list of tables
-- and views within the default schema of this database
-- (Use \help to learn more options)

\list

# Rank Countries by Total USD Spent in Renewable RD&D

In [ ]:
SELECT *
FROM COUNTRY_BUDGETS_SUMMARY
WHERE TIME > 2015
AND PRODUCT = 'RDDUSD'
AND FLOW NOT IN ('FOSSILFUEL', 'UNALLOC', 'TOTAL', 'MEMONLC')
AND COUNTRY NOT LIKE 'EU'
ORDER BY VALUE DESC;

In [ ]:
SELECT *
FROM COUNTRY_BUDGETS_SUMMARY
WHERE TIME > 2015
AND PRODUCT = 'RDDUSD'
AND FLOW NOT IN ('FOSSILFUEL', 'UNALLOC', 'TOTAL', 'MEMONLC')
AND COUNTRY NOT LIKE 'EU'
ORDER BY VALUE DESC;

In [ ]:
SELECT *
FROM COUNTRY_BUDGETS_SUMMARY
WHERE TIME > 2015
AND PRODUCT = 'RDDUSD'
AND FLOW NOT IN ('FOSSILFUEL', 'UNALLOC', 'TOTAL', 'MEMONLC')
AND COUNTRY NOT LIKE 'EU'
ORDER BY VALUE DESC;

## Create table ranked by Million USD spent in RD&D

In [ ]:
# Changing data type to float
totalrenew['VALUE'] = pd.to_numeric(totalrenew['VALUE'], errors='coerce')
totalrenew['VALUE'] = totalrenew['VALUE'].round(2)

# Group by 'COUNTRY' and calculate the total spending
country_spending = totalrenew.groupby('COUNTRY')['VALUE'].sum().reset_index()

# Sort the DataFrame by 'VALUE' in descending order and assign ranks
country_spending['RANK'] = country_spending['VALUE'].rank(ascending=False, method='dense')

country_spending.sort_values('RANK', ascending=True)

# Rank Countries by Total USD Spent in Fossil Fuels


In [ ]:
SELECT *
FROM COUNTRY_BUDGETS_SUMMARY
WHERE TIME > 2015
AND PRODUCT = 'RDDUSD'
AND FLOW = 'FOSSILFUEL'
AND COUNTRY NOT LIKE 'EU'
ORDER BY PRODUCT DESC;

## Create table ranked by Million USD spent in FossilFuels

In [ ]:
totalfossil['VALUE'] = pd.to_numeric(totalfossil['VALUE'], errors='coerce')
totalfossil['VALUE'] = totalfossil['VALUE'].round(2)

# Group by 'COUNTRY' and calculate the total spending
dirty_countries = totalfossil.groupby('COUNTRY')['VALUE'].sum().reset_index()

# Sort the DataFrame by 'VALUE' in descending order and assign ranks
dirty_countries['RANK'] = dirty_countries['VALUE'].rank(ascending=False, method='dense')

dirty_countries.sort_values('RANK', ascending=True)

## Per GDP Data

In [ ]:
SELECT *
FROM PERGDP
WHERE TIME > 2015;

In [ ]:
# pergdp['VALUE'] = pd.to_numeric(pergdp['VALUE'], errors='coerce')
# pergdp['VALUE'] = pergdp['VALUE'].round(2)

# pergdp['TIME'] = pd.to_datetime(pergdp['TIME'])

# plt.figure(figsize=(10, 6))

# sns.set_style("whitegrid")
# sns.lineplot(x="TIME", y="VALUE", hue="COUNTRY", data=pergdp)
# plt.show()



# Fossil Fuel Subsidies

In [ ]:
SELECT *
FROM Fossils.fossil_fuel_subsidies_gdp;

12.c.1 - Fossil-fuel subsidies (consumption and production) as a proportion of total GDP (%) - ER_FFS_CMPT_GDP

In [ ]:
# Rename column
fossilsubgdp.rename(columns={'12.c.1 - Fossil-fuel subsidies (consumption and production) as a proportion of total GDP (%) - ER_FFS_CMPT_GDP': '%Total_GDP'}, inplace=True)

### Since the country names do not match between the fossilsub and totalfossil dataframes, fuzzywuzzy is used to match the names using string matching

In [ ]:
# Uppercase the Country name from fossilsub table
fossilsubgdp['Entity'] = fossilsubgdp['Entity'].str.upper().str.strip()

# Get a master list of all unique Countries from totalfossil dataframe
# Used to match with countries in fossilsub table
unique_countries = pd.Series(totalfossil['COUNTRY'].unique()).sort_values()


def standardize_name(name, choices):
    """
    Return a standardized name based on a list of choices and a given name.

    Parameters
    ----------
    name : str
        The name to be standardized.
    choices : List[str]
        A list of possible choices for the standardized name.

    Returns
    -------
    str
        The standardized name from the list of choices that is most similar to the given name.

    Notes
    -----
    This function uses fuzzy string matching to identify similar names in the choices list and
    chooses the name with the minimum length as the standardized name. The matching threshold for
    fuzzy string matching is set at 90% using the token set ratio method from the `fuzzywuzzy` library.
    """
    # create a list of all names similar to the current name
    similar_names = [n for n in choices if fuzz.token_set_ratio(name, n) > 90]
    if not similar_names:
        # return the original name if no names with a similarity ratio greater than 90% are found
        return name
    # sort similar names alphabetically
    similar_names = sorted(similar_names)
    # get name with minimum length
    standardized_name = min(similar_names, key=len)

    return standardized_name



fossilsubgdp['NORM_COUNTRY'] = fossilsubgdp['Entity'].apply(standardize_name, choices=unique_countries)

In [ ]:
fossilsubgdp.head()

In [ ]:
# Select only the countries found in totalfossil
fossilsubgdp = fossilsubgdp[fossilsubgdp['NORM_COUNTRY'].isin(unique_countries)]


# Select only Canada
fossilsubgdpcan = fossilsubgdp[(fossilsubgdp['NORM_COUNTRY'] == 'CANADA')]



In [ ]:
plt.figure(figsize=(10,6))

sns.lineplot(data=fossilsubgdpcan, x='Year', y='%Total_GDP', hue='NORM_COUNTRY')
plt.xticks(fossilsubgdpcan['Year'])
plt.show()

In [ ]:
fossilsubgdp[fossilsubgdp['NORM_COUNTRY'].str.contains('AUSTRALI')]

In [ ]:
SELECT *
FROM INDICS;

In [ ]:
-- -- Joining RD&D per thousand units of GDP to INDICS
-- -- INDICS table has exchange currency by country and year.
-- -- Because Total RD&D is in nominal national currencies divided by
-- -- GDP in nominal national currencies at market prices and
-- -- volumes, expressed in thousand units of GDP

-- SELECT PERGDP.*, INDICS.PRODUCT, INDICS.VALUE
-- FROM PERGDP
-- LEFT JOIN (
--     SELECT COUNTRY, PRODUCT, TIME, VALUE
--     FROM INDICS
--     WHERE PRODUCT = 'USEXRMEI'
-- ) AS INDICS
-- ON PERGDP.COUNTRY = INDICS.COUNTRY AND PERGDP.TIME = INDICS.TIME;